# Reducing Bias in Machine Learning Models - Data Cleaning

#### Kristen Lo - BrainStation
---

### Table of Contents
- [Introduction](#intro)
- [Part 0: Cleaning the Data](#clean)
    - 1.1: [Housekeeping](#housekeeping)
    - 1.2: [Deleting Columns](#col)
    - 1.3: [Deleting Rows](#rows)
    - 1.4: [Handling Duplicates](#duplicates)
    - 1.5: [Data Type Conversion](#type)
    - 1.6: [Handling Missing Data](#missing) 
    - 1.7: [Categorizing Columns](#cat) 
- [Conclusion](#conc)


---
### <a id = 'intro'></a> Introduction

In this notebook, we will be cleaning the data received so that it can be ready for further analysis.

Data was sourced from all adult Emergency Department visits from March 2014 - July 2017 from one academic and two community emergency rooms, apart from the Yale New Haven Health system. These visits resulted in either admission to their respective hospital or discharge. 

There are a total of 972 variables that we extracted per patient visit from 560,486 patient visits. 

Courtesy of:
 "Hong WS, Haimovich AD, Taylor RA (2018) Predicting hospital admission at emergency department triage using machine learning. PLoS ONE 13(7): e0201016." (https://doi.org/10.1371/journal.pone.0201016)




-----

## <a id = 'clean'></a> Part 0: Cleaning the Data

---
#### <a id = 'housekeeping'></a> 1.1 HouseKeeping 

For housekeeping, we will import all necessary libraries and upload the data. In addition we will look at the data set from a very high level view. We want to know generally what we are working with and to familiarize ourself with the data before we start cleaning. 

Plan of attack: 
- Import libraries needed and import the file 
- View the data and find out it's shape 
- Understand unique entries to the columns 

Importing all necessary libraries

In [1]:
!pip install pyreadr -q
import pandas as pd
import pyreadr
import numpy as np
import matplotlib.pyplot as plt

The dataset is stored as an .rdata file, let's import it so cleaning can begin. The relative path won't work so the full path is needed. 

In [3]:
result = pyreadr.read_r('Data/5v_cleandf.rdata')

Let's take a quick look at our data to see what we are working with.

In [ ]:
health_data = result['df']

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)

health_data.head(20)

dep_name esi age  gender           ethnicity  \
0         B   4  40    Male  Hispanic or Latino   
1         B   4  66    Male  Hispanic or Latino   
2         B   2  66    Male  Hispanic or Latino   
3         A   2  66    Male  Hispanic or Latino   
4         A   3  84  Female  Hispanic or Latino   
5         A   3  86  Female  Hispanic or Latino   
6         A   3  86  Female  Hispanic or Latino   
7         A   4  87  Female  Hispanic or Latino   
8         B   2  75    Male        Non-Hispanic   
9         B   2  75    Male        Non-Hispanic   
10        B   2  76    Male        Non-Hispanic   
11        B   2  77    Male        Non-Hispanic   
12        B   2  52    Male        Non-Hispanic   
13        B   3  53    Male  Hispanic or Latino   
14        B   3  53    Male  Hispanic or Latino   
15        B   2  53    Male  Hispanic or Latino   
16        B   3  53    Male  Hispanic or Latino   
17        B   2  53    Male  Hispanic or Latino   
18        B   1  53    Male  Hispanic or Latino   
19        A   3  33    Male  Hispanic or Latino   

                                         race     lang     religion  \
0                          White or Caucasian  English         None   
1   Native Hawaiian or Other Pacific Islander  English  Pentecostal   
2   Native Hawaiian or Other Pacific Islander  English  Pentecostal   
3   Native Hawaiian or Other Pacific Islander  English  Pentecostal   
4                                       Other    Other  Pentecostal   
5                                       Other    Other  Pentecostal   
6                                       Other    Other  Pentecostal   
7                                       Other    Other  Pentecostal   
8                          White or Caucasian  English         None   
9                          White or Caucasian  English         None   
10                         White or Caucasian  English         None   
11                         White or Caucasian  English         None   
12                                      Other  English     Catholic   
13                                      Other  English     Catholic   
14                                      Other  English     Catholic   
15                                      Other  English     Catholic   
16                                      Other  English     Catholic   
17                                      Other  English     Catholic   
18                                      Other  English     Catholic   
19                                      Other  English     Catholic   

        maritalstatus  employstatus insurance_status disposition arrivalmode  \
0              Single     Full Time            Other   Discharge     Walk-in   
1             Married  Not Employed       Commercial   Discharge         Car   
2             Married  Not Employed       Commercial   Discharge     Walk-in   
3             Married  Not Employed       Commercial   Discharge         Car   
4             Widowed       Retired         Medicare       Admit     Walk-in   
5             Widowed       Retired         Medicare   Discharge     Walk-in   
6             Widowed       Retired         Medicare       Admit         Car   
7             Widowed       Retired         Medicare   Discharge         Car   
8             Married       Retired         Medicare       Admit   ambulance   
9             Married       Retired         Medicare       Admit   ambulance   
10            Married       Retired         Medicare       Admit   ambulance   
11            Married       Retired         Medicare       Admit   ambulance   
12             Single     Full Time            Other   Discharge     Walk-in   
13  Significant Other      Disabled         Medicare       Admit   ambulance   
14  Significant Other      Disabled         Medicare       Admit   ambulance   
15  Significant Other      Disabled         Medicare       Admit     Walk-in   
16  Significant Other      Disabled         Medicare   Discharge   ambulance   

Looking at the data dictionary, we can see that columns are grouped into the following distinct categories: 
- `Demographics`: All demographic data related to patient
- `Triage Variables`: All vital signs recorded by the triage nurse
- `Hospital Usage`: How often the patient used the hospital, prior surgeries or hospitalizations
- `Chief Complaint`: What they visited the ER for
- `Past Medical History ` : Pre-existing conditions of patients
- `Outpatient Medications` : Number of outpatient medications taken divided up by type. 
- `Imaging` : Number of imaging procedures within one year of the current visit.
- `Historical Labs (Numberic)`: The last min, max, and median recorded value of all vitals taken in the ED within one year of the current visit 
- `Historical Labs (Categorical)` : The last value, number of positive results, and the total number of tests ordered for selected categorical labs ordered in the ED within one year of the current visit 

In [ ]:
health_data.size #to find out the number of rows 

544792392

In [ ]:
health_data.shape[1] #to find out the number of columns

972

As we can see, our data frame is 927 columns and ~550k rows. 

Now, let's try to see what exists within the columns of the data. 

In [ ]:
#Writing a loop to retireve all unique values for all of the columns
for column in health_data.columns:
    unique_vals = health_data[column].unique()
    print(f'Unique values in {column}: {unique_vals}')

Unique values in dep_name: ['B', 'A', 'C']
Categories (3, object): ['A', 'B', 'C']
Unique values in esi: ['4', '2', '3', '1', '5', NaN]
Categories (5, object): ['1', '2', '3', '4', '5']
Unique values in age: [40 66 84 86 87 75 76 77 52 53 33 49 50 51 45 46 35 64 31 32 41 43 21 22
 56 20 24 25 26 27 62 54 23 90 91 92 67 37 38 39 55 48 82 28 29 57 69 70
 71 44 60 95 34 65 47 19 42 63 61 72 18 30 59 73 58 81 88 80 85 68 78 74
 83 89 36 100 98 99 94 93 79 101 102 96 97 106 107 103 104 105 nan 108]
Unique values in gender: ['Male', 'Female']
Categories (2, object): ['Female', 'Male']
Unique values in ethnicity: ['Hispanic or Latino', 'Non-Hispanic', 'Patient Refused', 'Unknown']
Categories (4, object): ['Hispanic or Latino', 'Non-Hispanic', 'Patient Refused', 'Unknown']
Unique values in race: ['White or Caucasian', 'Native Hawaiian or Other Pacific Islander', 'Other', 'Black or African American', 'Patient Refused', 'Asian', 'Unknown', 'American Indian or Alaska Native', NaN]
Categories (8, 

Unique values in earlylabor: [0. 1.]
Unique values in ecodesadverseeffectsofmedicalcare: [0. 1.]
Unique values in ecodesadverseeffectsofmedicaldrugs: [0. 1.]
Unique values in ecodescutpierce: [0. 1.]
Unique values in ecodesdrowningsubmersion: [0. 1.]
Unique values in ecodesfall: [0. 1.]
Unique values in ecodesfirearm: [0. 1.]
Unique values in ecodesfireburn: [0. 1.]
Unique values in ecodesmachinery: [0.]
Unique values in ecodesmotorvehicletrafficmvt: [0. 1.]
Unique values in ecodesnaturalenvironment: [0. 1.]
Unique values in ecodesotherspecifiedandclassifiable: [0. 1.]
Unique values in ecodesotherspecifiednec: [0. 1.]
Unique values in ecodespedalcyclistnotmvt: [0. 1.]
Unique values in ecodesplaceofoccurrence: [0. 1.]
Unique values in ecodespoisoning: [0. 1.]
Unique values in ecodesstruckbyagainst: [0. 1.]
Unique values in ecodessuffocation: [0. 1.]
Unique values in ecodestransportnotmvt: [0. 1.]
Unique values in ecodesunspecified: [0. 1.]
Unique values in ectopicpreg: [0. 1.]
Unique va

From what we can see, there are already null values that exist within the data set. Let's look at the percentage of nulls per column. 

In [ ]:
#This will show the percentage of null values per column
health_data.isna().sum()/health_data.shape[0]*100

dep_name                                                0.000000
esi                                                     0.438370
age                                                     0.001963
gender                                                  0.000000
ethnicity                                               0.000000
race                                                    0.004639
lang                                                    0.000000
religion                                                0.000000
maritalstatus                                           0.000000
employstatus                                            0.000000
insurance_status                                        0.000000
disposition                                             0.000000
arrivalmode                                             3.904469
arrivalmonth                                            0.000000
arrivalday                                              0.000000
arrivalhour_bin          

As we can see, we have a few columns that have null values. 